### **autonomous_example_ros.py**

In [ ]:
#!/usr/bin/env python3
import rospy
import numpy as np
import math

from sensor_msgs.msg import PointCloud2
import sensor_msgs.point_cloud2 as pc2
from visualization_msgs.msg import Marker, MarkerArray
from nav_msgs.msg import Odometry
from fs_msgs.msg import ControlCommand

# PARAMETERS
max_throttle = 0.2
target_speed = 4.0
max_steering = 0.3
cones_range_cutoff = 7.0

latest_points = None
latest_odom = None

# ROS INITIALIZATION
rospy.init_node("fsds_autonomous_node")

pub_cones = rospy.Publisher("/fsds/cones", MarkerArray, queue_size=1)

# 공식 제어 토픽
pub_control = rospy.Publisher("/fsds/control_command", ControlCommand, queue_size=1)

rate = rospy.Rate(20)

# UTILITY FUNCTIONS
def distance(x1, y1, x2, y2):
    return math.sqrt((x1-x2)**2 + (y1-y2)**2)

def pointgroup_to_cone(group):
    xs = [p[0] for p in group]
    ys = [p[1] for p in group]
    return {'x': np.mean(xs), 'y': np.mean(ys)}

def find_cones(points):
    cones = []
    group = []
    for i in range(1, len(points)):
        if distance(points[i][0], points[i][1], points[i-1][0], points[i-1][1]) < 0.1:
            group.append(points[i])
        else:
            if len(group) > 0:
                cone = pointgroup_to_cone(group)
                if distance(0, 0, cone['x'], cone['y']) < cones_range_cutoff:
                    cones.append(cone)
            group = []
    return cones

def calculate_steering(cones):
    avg_y = np.mean([c['y'] for c in cones])
    return -max_steering if avg_y > 0 else max_steering

def calculate_throttle(odom):
    vx = odom.twist.twist.linear.x
    vy = odom.twist.twist.linear.y
    speed = math.sqrt(vx*vx + vy*vy)
    return max_throttle * max(1 - speed / target_speed, 0)

# ---------------------------
# CALLBACKS
# ---------------------------
def lidar_callback(msg):
    global latest_points
    pts = []
    for p in pc2.read_points(msg, field_names=("x", "y", "z"), skip_nans=True):
        pts.append([p[0], p[1], p[2]])
    latest_points = np.array(pts, dtype=float)

def odom_callback(msg):
    global latest_odom
    latest_odom = msg

# SUBSCRIBERS
rospy.Subscriber("/fsds/lidar/Lidar1", PointCloud2, lidar_callback)
rospy.Subscriber("/fsds/testing_only/odom", Odometry, odom_callback)

# MAIN LOOP
while not rospy.is_shutdown():

    if latest_points is None or latest_odom is None:
        rate.sleep()
        continue

    cones = find_cones(latest_points)

    # RViz markers
    marker_array = MarkerArray()
    for i, c in enumerate(cones):
        m = Marker()
        m.header.frame_id = "base_link"
        m.id = i
        m.type = Marker.SPHERE
        m.action = Marker.ADD
        m.pose.position.x = c['x']
        m.pose.position.y = c['y']
        m.pose.position.z = 0.2
        m.scale.x = 0.3
        m.scale.y = 0.3
        m.scale.z = 0.3
        m.color.r = 1.0
        m.color.g = 0.5
        m.color.b = 0.0
        m.color.a = 1.0
        marker_array.markers.append(m)

    pub_cones.publish(marker_array)

    # vehicle control
    if len(cones) > 0:
        steering = calculate_steering(cones)
        throttle = calculate_throttle(latest_odom)

        cmd = ControlCommand()
        cmd.steering = steering
        cmd.throttle = throttle
        cmd.brake = 0.0

        pub_control.publish(cmd)

    rate.sleep()
